# Building Cooling Loads Example
### PIP Summer School 2018
Phillippe Phanivong & Elpiniki Apostolaki – Iosifidou

## Background Problem

One of the challenges in the power grid is that we can't tell customers when they can use electricity. They are buying a product from the utility and that product is reliable electricity when they want it. However, maybe we can incentivize people to reduce the amount of power they are using at certain times. However, to do this we need to know how customers are using their electricity and which ones have the ability to shift their consumption to different times of day. This notebook explores the cooling loads of different buildings to see if we can use climate data to predict the energy consumption of commerical building customers.

In [ ]:
import os
import numpy as np
import pandas as pd
import sklearn as sk

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt


# Where we're working out of
PROJECT_ROOT_DIR = "."
FOLDER_ID = "building_lab"
DATA_FOLDER = "Sacramento Building Load Models"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", FOLDER_ID)
#A function to save figures easily
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [ ]:
try: 
    os.mkdir(FOLDER_ID)
except:
    print("Folder already exists")

## Import and clean the data

In [ ]:
def load_tmy3_data(climate_data, data_path=DATA_FOLDER):
    csv_path = os.path.join(data_path, climate_data)
    return pd.read_csv(csv_path,skiprows=[0], parse_dates=True)

def load_building_data(building_data, data_path=DATA_FOLDER):
    csv_path = os.path.join(data_path, building_data)
    return pd.read_csv(csv_path)


In [ ]:
tmy3_data = load_tmy3_data("CA-Sacramento_Metro.tmy3")


In [ ]:
# Columns we want 
tmy3_data.head(15)

In [ ]:
#Strips the year out of the first column
tmy3_data.iloc[:,0] = tmy3_data.iloc[:,0].str.slice(0,5)

#Important categories
tmy3_data = tmy3_data.iloc[:,[0,1,31,37,40,43,46]]

#Rename columns
tmy3_data = tmy3_data.rename(columns={'Date (MM/DD/YYYY)':'date', 'Time (HH:MM)':'time',
                           'Dry-bulb (C)': 'temp', 'RHum (%)':'humidity',
                          'Pressure (mbar)': 'pressure', 'Wdir (degrees)': 'wind_dir',
                          'Wspd (m/s)':'wind_speed'})

In [ ]:
tmy3_data.head()

In [ ]:
#Notice describe() only shows the columns with integers or floats
tmy3_data.describe()

In [ ]:
tmy3_data.info()

In [ ]:
tmy3_data.hist(bins=50, figsize=(20,15))
#save_fig("attribute_histogram_plots")
plt.show()

In [ ]:
#Building energy for a Sacramento Metro Airport TMY3 Medium Office building
building_energy_data = load_building_data("RefBldgMediumOfficeNew2004_7.1_5.0_3B_USA_CA_LOS_ANGELES.csv")

In [ ]:
building_energy_data.head()

In [ ]:
#The combined cooling energy with matching date and time from the feature set
combined_set = pd.DataFrame(tmy3_data)
combined_set = combined_set.join(building_energy_data.iloc[:,3])
combined_set = combined_set.rename(columns={'Cooling:Electricity [kW](Hourly)':'cooling_load'})
combined_set.head()

In [ ]:
combined_set.hist(bins=50, figsize=(20,15))
#save_fig("attribute_histogram_plots")
plt.show()

In [ ]:
date_cat = combined_set['date']
date_cat_encoded, date_categories = date_cat.factorize()
time_cat = combined_set['time']
time_cat_encoded, time_categories = time_cat.factorize()

In [ ]:
combined_set = pd.DataFrame({'date_cat':date_cat_encoded,'time_cat':time_cat_encoded})
combined_set = pd.concat([combined_set,tmy3_data.iloc[:,2:7],building_energy_data.iloc[:,3]],axis=1)
combined_set = combined_set.rename(columns={'Cooling:Electricity [kW](Hourly)':'cooling_load'})

In [ ]:
combined_set.describe()

In [ ]:
combined_set.hist(bins=50, figsize=(20,15))
#save_fig("attribute_histogram_plots")
plt.show()

### Inspect the data

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(combined_set, figsize = (12,8))

In [ ]:
#attributes = ['cooling_load', 'temp', 'humidity', 'pressure']
#scatter_matrix(combined_set[attributes], figsize = (12,8))

In [ ]:
corr_matrix = combined_set.corr()
corr_matrix["cooling_load"]

### Split the Data

In [ ]:
from sklearn.model_selection import train_test_split

train_set,test_set = train_test_split(combined_set, test_size=0.2, random_state=42)


In [ ]:
train_set_safe = train_set.copy()
corr_matrix = train_set.corr()
corr_matrix["cooling_load"]

In [ ]:
train_set.plot(kind='scatter', x = 'temp', y = 'cooling_load', alpha = 0.1)


In [ ]:
feature_set_train = train_set.drop('cooling_load', axis =1)
label_set_train = train_set['cooling_load'].copy()

### Scale the data

In [ ]:
feature_set_train.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
scaled_feature_set_train = std_scaler.fit_transform(feature_set_train)

In [ ]:
min_max_scaler = sk.preprocessing.MinMaxScaler()
min_max_feature_set_train = min_max_scaler.fit_transform(feature_set_train)

# Training Regression Models

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

### Linear Regression

In [ ]:
lin_reg = LinearRegression()
#lin_reg.fit(feature_set_train, label_set_train)
#lin_reg.fit(min_max_feature_set_train, label_set_train)
lin_reg.fit(scaled_feature_set_train, label_set_train)

In [ ]:
lin_energy_predictions = lin_reg.predict(feature_set_train)
lin_mse = mean_squared_error(label_set_train, lin_energy_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
lin_energy_predictions = lin_reg.predict(min_max_feature_set_train)
lin_mse = mean_squared_error(label_set_train, lin_energy_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
lin_energy_predictions = lin_reg.predict(scaled_feature_set_train)
lin_mse = mean_squared_error(label_set_train, lin_energy_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

### Decision Tree Regression

In [ ]:
tree_reg = DecisionTreeRegressor(random_state = 42)
#tree_reg.fit(feature_set_train, label_set_train)
#tree_reg.fit(min_max_feature_set_train, label_set_train)
tree_reg.fit(scaled_feature_set_train, label_set_train)

In [ ]:
tree_energy_predictions = tree_reg.predict(feature_set_train)
tree_mse = mean_squared_error(label_set_train, tree_energy_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
tree_energy_predictions = tree_reg.predict(min_max_feature_set_train)
tree_mse = mean_squared_error(label_set_train, tree_energy_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
tree_energy_predictions = tree_reg.predict(scaled_feature_set_train)
tree_mse = mean_squared_error(label_set_train, tree_energy_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

### Plot Results

In [ ]:
# Make the plot.
plt.clf()
plt.plot(feature_set_train["temp"], label_set_train, 'o', color='grey', label = 'Actual Output')
plt.plot(feature_set_train["temp"], lin_energy_predictions, 'o', color='red', label = 'Predicted Output')
plt.xlabel("temperature")
plt.ylabel("Enegy")
plt.title("Linear Regression - Training Set")
plt.legend()
#plt.savefig("linear_regression_train_set.png")

In [ ]:
# Make the plot.
plt.clf()
plt.plot(feature_set_train["humidity"], label_set_train, 'o', color='grey', label = 'Actual Output')
plt.plot(feature_set_train["humidity"], tree_energy_predictions, 'o', color='red', label = 'Predicted Output')
plt.xlabel("Humidity")
plt.ylabel("Enegy")
plt.title("Decision Tree Regression - Training Set")
plt.legend()
plt.ylim((-1, 120))
plt.xlim((-1,120))
#plt.savefig("decision_tree_regression_train_set.png")

### Cross-Validate

In [ ]:
from sklearn.model_selection import cross_val_score
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
lin_scores = cross_val_score(lin_reg, scaled_feature_set_train, label_set_train,
                             scoring= "neg_mean_squared_error", cv = 10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

In [ ]:
scores = cross_val_score(tree_reg, scaled_feature_set_train, label_set_train,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)
display_scores(tree_rmse_scores)

### Polynomial Regression

In [ ]:
from sklearn.preprocessing import PolynomialFeatures 
poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_poly_set_train = poly_features.fit_transform(min_max_feature_set_train)
poly_reg = LinearRegression()
poly_reg.fit(X_poly_set_train, label_set_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
std_scaler = StandardScaler()
poly_reg_pipe = LinearRegression()
poly_pipe = Pipeline([("poly_features", poly_features), ("std_scaler", std_scaler),("poly_reg", poly_reg_pipe),])
poly_pipe.fit(feature_set_train, label_set_train)

In [ ]:
poly_energy_predictions = poly_reg.predict(X_poly_set_train)
poly_mse = mean_squared_error(label_set_train, poly_energy_predictions)
poly_rmse = np.sqrt(poly_mse)
poly_rmse

In [ ]:
poly_energy_predictions = poly_pipe.predict(feature_set_train)
poly_mse = mean_squared_error(label_set_train, poly_energy_predictions)
poly_rmse = np.sqrt(poly_mse)
poly_rmse

In [ ]:
#Try with different degrees of freedom
poly_scores = cross_val_score(poly_pipe, feature_set_train, label_set_train,
                             scoring= "neg_mean_squared_error", cv = 10)
poly_rmse_scores = np.sqrt(-poly_scores)
display_scores(poly_rmse_scores)

In [ ]:
from sklearn.linear_model import Ridge
pipe_poly_features = PolynomialFeatures(degree=6, include_bias=False)
pipe_std_scaler = StandardScaler()
pipe_ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
ridge_pipe = Pipeline([("poly_features", pipe_poly_features), ("std_scaler", pipe_std_scaler),("poly_reg", pipe_ridge_reg),])
ridge_pipe.fit(feature_set_train, label_set_train)

In [ ]:
ridge_energy_predictions = ridge_pipe.predict(feature_set_train)
ridge_mse = mean_squared_error(label_set_train, ridge_energy_predictions)
ridge_rmse = np.sqrt(ridge_mse)
ridge_rmse

In [ ]:
#Try with different degrees of freedom
ridge_scores = cross_val_score(ridge_pipe, feature_set_train, label_set_train,
                             scoring= "neg_mean_squared_error", cv = 10)
ridge_rmse_scores = np.sqrt(-ridge_scores)
display_scores(ridge_rmse_scores)

### Random Forrest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state = 42)
forest_reg.fit(scaled_feature_set_train, label_set_train)

In [ ]:
forest_energy_predictions = forest_reg.predict(scaled_feature_set_train)
forest_mse = mean_squared_error(label_set_train, forest_energy_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
forest_scores = cross_val_score(forest_reg, scaled_feature_set_train, label_set_train,
                                 scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
pd.Series(forest_rmse_scores).describe()

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3,10,30,50,100], 'max_features': [2,4,6]},
    {'bootstrap': [False], 'n_estimators':[3,10], 'max_features': [2,3,4]},
    ]

forest_reg_grid = RandomForestRegressor(random_state =42)
grid_search = GridSearchCV(forest_reg_grid, param_grid, cv=5,
                           scoring = "neg_mean_squared_error" , return_train_score=True)
grid_search.fit(scaled_feature_set_train, label_set_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=7),
    }
forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions = param_distribs,
                                n_iter=50, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(scaled_feature_set_train, label_set_train)



In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

In [ ]:
feature_importances = rnd_search.best_estimator_.feature_importances_
feature_importances

# Test Data

In [ ]:
final_model=rnd_search.best_estimator_

X_test = test_set.drop('cooling_load', axis =1)
y_test = test_set['cooling_load'].copy()

X_test_prepared = std_scaler.fit_transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse